In [1]:
from cosmikyu import gan, config, model
from cosmikyu import nn as cnn
import numpy as np
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torch
import mlflow
import torchsummary

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
STanh = cnn.ScaledTanh(30., 2./30.)
LF = cnn.LinearFeature(4,4, bias=True)
shape = (5,128,128)
PIXGAN = gan.PIXGAN_WGP("sehgal_pixgan_251020", shape, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=False, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=5, kernal_size=4, stride=2,
                          padding=1, output_padding=0, gen_act=[LF,STanh], nin_channel=1, nout_channel=4, nthresh_layer_gen=3, nthresh_layer_disc=0, dropout_rate=0)

print("====pixgan======")
torchsummary.summary(PIXGAN.discriminator, shape, batch_size=1, device="cpu")
torchsummary.summary(PIXGAN.generator, (1,)+shape[-2:], batch_size=1, device="cpu")

/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.M

[WARNING] You have a CUDA device. You probably want to run with CUDA enabled
====pixgan======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 64, 64, 64]           5,184
         LeakyReLU-2            [1, 64, 64, 64]               0
            Conv2d-3           [1, 128, 32, 32]         131,200
         LeakyReLU-4           [1, 128, 32, 32]               0
            Conv2d-5           [1, 256, 16, 16]         524,544
         LeakyReLU-6           [1, 256, 16, 16]               0
            Conv2d-7             [1, 512, 8, 8]       2,097,664
         LeakyReLU-8             [1, 512, 8, 8]               0
            Conv2d-9            [1, 1024, 4, 4]       8,389,632
        LeakyReLU-10            [1, 1024, 4, 4]               0
          Reshape-11                 [1, 16384]               0
           Linear-12                     [1, 1]          16,385
Total par

In [95]:


shape = (6,128,128)
genin_shape = (3,128,128)
discin_shape = (6,128,128)
unet_gen = model.UNET_Generator(
    genin_shape,
    nconv_layer=3,
    nconv_fc=64,
    ngpu=1,
    kernal_size=5,
    stride=2,
    padding=2,
    output_padding=1,
    normalize=True,
    activation=[torch.nn.Tanh()],
    nin_channel = 3,
    nout_channel = 3,
    nthresh_layer = 4
).to(device="cpu")

unet_disc = model.UNET_Discriminator(
    shape=discin_shape,
    nconv_layer=4,
    nconv_fc=64,
    ngpu=1,
    kernal_size=5,
    stride=2,
    padding=2,
    normalize=False,
    nthresh_layer=2
).to(device="cpu")


print("====dc_wgp======")
torchsummary.summary(unet_disc, discin_shape, device="cpu")
torchsummary.summary(unet_gen, genin_shape, device="cpu")
#print(type(unet_disc.model))
#torchsummary.summary(DCGAN_WGP.generator, (latent_dim,))

====dc_wgp======
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,664
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         204,928
         LeakyReLU-4          [-1, 128, 32, 32]               0
            Conv2d-5          [-1, 256, 16, 16]         819,456
         LeakyReLU-6          [-1, 256, 16, 16]               0
            Conv2d-7            [-1, 512, 8, 8]       3,277,312
         LeakyReLU-8            [-1, 512, 8, 8]               0
            Conv2d-9            [-1, 512, 8, 8]       6,554,112
      BatchNorm2d-10            [-1, 512, 8, 8]           1,024
        LeakyReLU-11            [-1, 512, 8, 8]               0
           Conv2d-12            [-1, 512, 8, 8]       6,554,112
      BatchNorm2d-13            [-1, 512, 8, 8]           1,024
        LeakyReLU-14  

In [104]:
shape = (3,128,128)
genin_shape = (3,224,224)
discin_shape = (6,128,128)
unet_gen = model.ResUNET_Generator(
    genin_shape,
    nconv_layer=3,
    nconv_fc=64,
    ngpu=1,
    activation=[torch.nn.Tanh()],
    nin_channel = 3,
    nout_channel = 3,
    nthresh_layer = 1,
).to(device="cpu")

torchsummary.summary(unet_gen, genin_shape, device="cpu")


512 256 0 1
256 128 1 2
128 64 2 3
64 32 3 4
torch.Size([2, 256, 56, 56]) torch.Size([2, 256, 56, 56])
torch.Size([2, 128, 112, 112]) torch.Size([2, 128, 112, 112])
torch.Size([2, 64, 224, 224]) torch.Size([2, 64, 224, 224])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
            Conv2d-5         [-1, 64, 224, 224]           1,792
      ResUNetBlock-6         [-1, 64, 224, 224]               0
       BatchNorm2d-7         [-1, 64, 224, 224]             128
              ReLU-8         [-1, 64, 224, 224]               0
            Conv2d-9        [-1, 128, 112, 112]          73,856
      BatchNorm2d-10        [-1, 128, 112, 112]             256
      

In [91]:
unet_gen.model_dict['up0_int']

ResUNetUPInterface(
  (model): Sequential(
    (0): ConvTranspose2d(512, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  )
)

In [92]:
unet_gen.model_dict['up0']

ResUNetBlock(
  (model): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (imodel): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [29]:

shape = (3,128,128)
genin_shape = (3,128,128)
unet_gen = model.ResUNetDown(
    in_filters=3, 
    out_filters=64, 
    outermost=True, 
    kernal_size=3,
    padding=1, ngpu=0, use_leaky=False
).to(device="cpu")

torchsummary.summary(unet_gen, genin_shape, device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [-1, 3, 128, 128]               6
              ReLU-2          [-1, 3, 128, 128]               0
            Conv2d-3         [-1, 64, 128, 128]           1,792
       BatchNorm2d-4         [-1, 64, 128, 128]             128
              ReLU-5         [-1, 64, 128, 128]               0
            Conv2d-6         [-1, 64, 128, 128]          36,928
            Conv2d-7         [-1, 64, 128, 128]           1,792
Total params: 40,646
Trainable params: 40,646
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 40.75
Params size (MB): 0.16
Estimated Total Size (MB): 41.09
----------------------------------------------------------------
